In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pareto

In [ ]:
# Generate a random number from a uniform distribution between 0 and 2π
def random_direction():    
    θ = np.random.uniform(0, 2*np.pi)
    return np.array([np.cos(θ),np.sin(θ)]) 

# Generate the length of time and space displacement
alpha = 1 # [0.5,1,1.5,2,3]
beta = 0.5 # [0.5,1,1.5]
Δt_min = 0.1
